# Classification cellules normales  : Machine Learning
**Avec Random Forest, classification cellules : basophil, eosinophil, erythroblast, ig, lymphocyte, monocyte, neutrophil et platelet**

Etude des images (4000) en format 255*255 en niveau de gris : 500 images par catégorie

Résultat servant de baseline pour l'optimisation

In [1]:
import pandas as pd
import cv2
import numpy as np

In [2]:
import utils_blood_cells as utils

In [8]:
# Récupère un échantillon de 4000 images de cellules normales : 500  par type
utils.change_path_root('../')
df_data_cells, data =  utils.get_mendeley_cells(size=4000, stratify_category = True)

In [9]:
#Cible : le type de cellule
target = df_data_cells['category'] 
target.value_counts()

neutrophil      500
eosinophil      500
ig              500
platelet        500
erythroblast    500
monocyte        500
basophil        500
lymphocyte      500
Name: category, dtype: int64

In [10]:
#Modèle baseline : Random Forest
from sklearn.ensemble import RandomForestClassifier 
from sklearn.model_selection  import train_test_split

#Séparation données entrainement, données test
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size= 0.20)
rf_clf = RandomForestClassifier(n_jobs=-1)

rf_clf.fit(X_train, y_train)

RandomForestClassifier(n_jobs=-1)

In [11]:
# Prédiction sur X_test
y_pred_test = rf_clf.predict(X_test)

In [12]:
# Evaluation de la prédiction
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_test)) 

              precision    recall  f1-score   support

    basophil       0.79      0.80      0.79        89
  eosinophil       0.81      0.83      0.82       100
erythroblast       0.89      0.89      0.89        95
          ig       0.69      0.66      0.67        96
  lymphocyte       0.87      0.89      0.88       120
    monocyte       0.77      0.77      0.77       102
  neutrophil       0.89      0.85      0.87       102
    platelet       0.99      1.00      0.99        96

    accuracy                           0.84       800
   macro avg       0.84      0.84      0.84       800
weighted avg       0.84      0.84      0.84       800



*Taux de prédiction de 84%, bon score de base*

Moins bon score sur la catégorie ig. Ce qui peut s'expliquer par un regroupement de sous catégorie dans cette catégorie.
A voir si doit subdiviser cette catégorie pour une meilleure classification

Score proche de 100% sur la catégorie platelet, qui peut s'expliquer par une petite taille de ces cellules, facilement identifiable

In [13]:
# Matrice de confusion
pd.crosstab(y_test, y_pred_test, rownames=['Realité'], colnames=['Prédiction'])

Prédiction,basophil,eosinophil,erythroblast,ig,lymphocyte,monocyte,neutrophil,platelet
Realité,,,,,,,,
basophil,71,1,0,5,6,4,2,0
eosinophil,0,83,1,3,0,9,3,1
erythroblast,2,0,85,2,4,0,2,0
ig,6,11,2,63,4,8,2,0
lymphocyte,4,1,4,3,107,1,0,0
monocyte,7,3,0,11,0,79,2,0
neutrophil,0,4,4,4,2,1,87,0
platelet,0,0,0,0,0,0,0,96


La catégorie IG semble vraiment posé problème pour l'identification, mauvaise précision ainsi que  mauvais rappel

Confusion entre basophil et lymphocyte ainsi qu'entre eosinophil et monocyte


**Cross-validation pour évaluer la stabilité du modèle**

In [14]:
from sklearn.model_selection import cross_validate
cv_results = cross_validate(rf_clf, data, target, cv=5, return_train_score=True) 
scores = cv_results["test_score"]
print("La moyenne d'accuracy des données tests est: " 
      f"{scores.mean():.3f} +/- {scores.std():.3f}")


La moyenne d'accuracy des données tests est: 0.817 +/- 0.015


La cross-validation montre une moyenne similaire des précisions des prédictions

Le modèle semble assez stable

### Modèle SVC

In [19]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(), SVC())
pipe.get_params()

{'memory': None,
 'steps': [('standardscaler', StandardScaler()), ('svc', SVC())],
 'verbose': False,
 'standardscaler': StandardScaler(),
 'svc': SVC(),
 'standardscaler__copy': True,
 'standardscaler__with_mean': True,
 'standardscaler__with_std': True,
 'svc__C': 1.0,
 'svc__break_ties': False,
 'svc__cache_size': 200,
 'svc__class_weight': None,
 'svc__coef0': 0.0,
 'svc__decision_function_shape': 'ovr',
 'svc__degree': 3,
 'svc__gamma': 'scale',
 'svc__kernel': 'rbf',
 'svc__max_iter': -1,
 'svc__probability': False,
 'svc__random_state': None,
 'svc__shrinking': True,
 'svc__tol': 0.001,
 'svc__verbose': False}

In [21]:
grid_model = GridSearchCV(estimator=pipe, 
                          param_grid={'svc__C': [0.1, 1, 10], 'svc__kernel': ('linear', 'rbf')}) 

grid = grid_model.fit(X_train,y_train)

In [24]:
grid.best_params_

{'svc__C': 10, 'svc__kernel': 'rbf'}

In [23]:
# Prédiction sur X_test
y_pred_test = grid.predict(X_test)
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

    basophil       0.73      0.78      0.75        89
  eosinophil       0.74      0.75      0.75       100
erythroblast       0.83      0.84      0.84        95
          ig       0.61      0.62      0.62        96
  lymphocyte       0.86      0.84      0.85       120
    monocyte       0.75      0.76      0.76       102
  neutrophil       0.89      0.79      0.84       102
    platelet       0.99      0.99      0.99        96

    accuracy                           0.80       800
   macro avg       0.80      0.80      0.80       800
weighted avg       0.80      0.80      0.80       800



> Modèle SVC avec une précision légérement inférieure